In [24]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from piston_1 import DMDPiston
from waveoptics.plots.plots import complex_imshow
from dmdholo.fields_propagation.fourier_filter import fourier_filter
from dmdholo.holograms.dmd_holograms import parallel_lee,  orthogonal_lee, holo_SP, holo_Haskell, holo_SP_optimized
from dmdholo.holograms.generate_LUT import generate_lut
from dmdholo.plotting.plot_funs import colorize
# from thorlabscam.camera import ThorlabsCamera


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Prepare camera

In [3]:
# # cam init
# cam = ThorlabsCamera(serial_number='24140', name='CAM_1')
# cam.setup(
#     exposure_time_us=1500,
#     roi_xy_size=(304,304),# (W,H)
#     roi_xy_upper_left=(1292,1560),#roi_Origine_X,Y_
#     frames_per_trigger_zero_for_unlimited= 1,
#     operation_mode=1,trigger_polarity = '1',
#     ) 


In [ ]:
# cam.dispose()

# LUT Calculation 

## Px=4

In [25]:
px = 4
nuvec = np.array((1/px,1/px**2)) 
SA_filter = 1.0*np.linalg.norm(nuvec)/6
f_val, px_comb, lut = generate_lut('sp', px)

# HOLO (Actuators) 

## Generation 

In [29]:
DMD_nSizeY=768
DMD_nSizeX=1024
n_actu_1D = 16
roi_size=256
# Initialisation du DMD avec une région d'intérêt (ROI)
dmd = DMDPiston(n_actu_1D, roi_size=roi_size, roi_shifts=(370, 520))  # H,V (400,520)

# Génération d'hologrammes
nbr_holos = 10000
Holos_Actu = []
maps=[]
generated_fields = set()
for i in range(nbr_holos):  # Correction de `np.(nbr_holos)` en `range(nbr_holos)`
    # Générer des champs avec des amplitudes et phases aléatoires
    dmd.generate_fields(n_actu_1D, rand_amp=True, rand_phi=True,phi_clx=None,phi=None)
    target_field = np.copy(dmd.field_matrix)
    maps.append(dmd.field)
    
    # field_hash = hash(dmd.actus.tobytes())
    # if field_hash in generated_fields:
    #     print(f"Duplicate found! in {i}")
    # generated_fields.add(field_hash)
    
    target_field /= np.max(np.abs(target_field))
    # Conversion en hologramme
    holo_sp = holo_SP(target_field, lut, px_comb).astype(np.uint8)
    # holo_sp = holo_SP_optimized(target_field, lut, px_comb).astype(np.uint8)
    # holo_sp2 = holo_SP(target_field, lut, px_comb).astype(np.uint8)
    
    holo_sp=holo_sp*(2**8-1)
    # holo_sp2=holo_sp2*(2**8-1)

    Holos_Actu.append(holo_sp)
    if i % 100 == 0:
        print("Progress: Generated holograms =", i)

Progress: Generated holograms = 0
Progress: Generated holograms = 100
Progress: Generated holograms = 200
Progress: Generated holograms = 300
Progress: Generated holograms = 400
Progress: Generated holograms = 500
Progress: Generated holograms = 600
Progress: Generated holograms = 700
Progress: Generated holograms = 800
Progress: Generated holograms = 900
Progress: Generated holograms = 1000
Progress: Generated holograms = 1100
Progress: Generated holograms = 1200
Progress: Generated holograms = 1300
Progress: Generated holograms = 1400
Progress: Generated holograms = 1500
Progress: Generated holograms = 1600
Progress: Generated holograms = 1700
Progress: Generated holograms = 1800
Progress: Generated holograms = 1900
Progress: Generated holograms = 2000
Progress: Generated holograms = 2100
Progress: Generated holograms = 2200
Progress: Generated holograms = 2300
Progress: Generated holograms = 2400
Progress: Generated holograms = 2500
Progress: Generated holograms = 2600
Progress: Gen

## Saving the holos in a folder 

In [32]:
np.savez(rf'C:\Users\saucourt\Downloads\holo_Acuators_size={nbr_holos}_1D_{n_actu_1D}_ROI{roi_size}.npz',
         Holos_Actu=np.array(Holos_Actu),  phase_maps_set=generated_fields)
